In [ ]:
!python -V

In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [5]:
df = pd.read_parquet('data/green_tripdata_2021-01.parquet')

df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [ ]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

In [ ]:
sns.displot(y_pred, label='prediction')
sns.displot(y_train, label='actual')

plt.legend()

In [ ]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

In [ ]:
len(df_train), len(df_val)

In [ ]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [ ]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

#### Homework 1

In [5]:
data = pd.read_parquet("data/yellow_tripdata_2022-01.parquet")

In [6]:
# Question 1
len(data.columns)

19

In [7]:
data.columns
data["duration"] = data["tpep_dropoff_datetime"] - data["tpep_pickup_datetime"]
data.duration = data.duration.apply(lambda td: td.total_seconds() / 60)


In [8]:
# Question 2
data.duration.std()

46.44530513776802

In [9]:
beforenum = len(data["duration"])
beforenum

2463931

In [10]:
data = data[(data.duration >= 1) & (data.duration <= 60)]


In [11]:
afternum = len(data["duration"])
afternum

2421440

In [12]:
# Question 3
(afternum / beforenum)

0.9827547930522406

In [21]:
categorical = ['PULocationID', 'DOLocationID']
data[categorical] = data[categorical].astype(str)
data_dict = data[categorical].to_dict(orient='records')


In [22]:
dv = DictVectorizer()
X_train = dv.fit_transform(data_dict)

In [24]:
# Question 4
X_train.shape[1]

515

In [25]:
target_var = 'duration'
y_train = data[target_var].values

In [26]:
y_train

array([17.81666667,  8.4       ,  8.96666667, ..., 11.        ,
       12.05      , 27.        ])

In [27]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [28]:
y_pred = lr.predict(X_train)

In [29]:
# Question 5
mean_squared_error(y_train, y_pred, squared=False)

6.986190799978604

In [5]:
# Question 6
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

valdata = read_dataframe('data/yellow_tripdata_2022-02.parquet')


In [ ]:
with open('models/hw1.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

# can't read data all at once, so I will clear the previous data and load 2022 February Yellow taxi data


In [6]:
load_model = pickle.load(open('models/hw1.bin', 'rb'))

In [7]:
y_val = valdata["duration"].values
valdata_dict = valdata[['PULocationID', 'DOLocationID']].to_dict(orient='records')
dv_val = load_model[0]
X_train = dv_val.transform(valdata_dict)

In [8]:
X_train

<2918187x515 sparse matrix of type '<class 'numpy.float64'>'
	with 5836368 stored elements in Compressed Sparse Row format>

In [10]:
# Question 6
y_pred = load_model[1].predict(X_train)
mean_squared_error(y_val, y_pred, squared=False)

7.786407313028804